In [1]:
import pickle as pkl
from adet.utils.visualize_niigz import *
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path as pa

In [2]:
# fname = '/home/hynx/regis/Recursive-Cascaded-Networks/evaluate/Jul15-1755-model-2500.pkl'
# fname = '/home/hynx/regis/Recursive-Cascaded-Networks/evaluate/Jul20-1506-model-1500.pkl'
# fname = '/home/hynx/regis/Recursive-Cascaded-Networks/evaluate/Jul20-1619-model-3500.pkl'
# fname = '/home/hynx/regis/Recursive-Cascaded-Networks/evaluate/Jul26-1518-model-4500.pkl'
# fname = '/home/hynx/regis/Recursive-Cascaded-Networks/evaluate/Aug08-1440-model-5560.pkl'
# fname = '/home/hynx/regis/Recursive-Cascaded-Networks/evaluate/Aug08-1815-model-5700-lits.pkl'
fname = '/home/hynx/regis/Recursive-Cascaded-Networks/evaluate/VTN-3-liver-model-99500.pkl'
dct = pkl.load(open(fname, 'rb'))
print(dct.keys())
# print(dct['dices'])

dict_keys(['pt_mask', 'landmark_dists', 'jaccs', 'dices', 'jacobian_det', 'warped_moving', 'real_flow', 'warped_seg_moving', 'warped_moving_0', 'warped_seg_moving_0', 'real_flow_0', 'warped_moving_1', 'warped_seg_moving_1', 'real_flow_1', 'warped_moving_2', 'warped_seg_moving_2', 'real_flow_2', 'warped_moving_3', 'warped_seg_moving_3', 'real_flow_3', 'id1', 'id2', 'seg1', 'seg2', 'img1', 'img2'])


In [39]:
dir_name = 'normal'
(pa('images')/dir_name).mkdir(exist_ok=True)
# include img1,2 seg1,2 warped_moving
keys = ['img1', 'img2', 'seg1', 'seg2', 'warped_moving']

k = 'warped_seg_moving'
p = np.maximum((dct[k][...,0,1]>0.5)*2,(dct[k][...,0,0]>0.5))

seg_pairs = list(zip(*[dct[k][...,0] for k in keys], p))
keys.append(k)

vis_id = -2

def vis_basic():
    for s in list(seg_pairs)[vis_id:vis_id+1]:
        for i in range(len(s)):
            print(s[i].shape)
            # print(np.unique(s[i]))
            visulize_3d(s[i], inter_dst=3, save_name=f'images/{dir_name}/{keys[i]}.jpg')
            print(keys[i])
        break

vis_basic()

(128, 128, 128)
Visualizing img with shape and type: torch.Size([43, 1, 128, 128]) torch.float32
img1
(128, 128, 128)
Visualizing img with shape and type: torch.Size([43, 1, 128, 128]) torch.float32
img2
(128, 128, 128)
Visualizing img with shape and type: torch.Size([43, 1, 128, 128]) torch.float32
seg1
(128, 128, 128)
Visualizing img with shape and type: torch.Size([43, 1, 128, 128]) torch.float32
seg2
(128, 128, 128)
Visualizing img with shape and type: torch.Size([43, 1, 128, 128]) torch.float32
warped_moving
(128, 128, 128)
Visualizing img with shape and type: torch.Size([43, 1, 128, 128]) torch.float32
warped_seg_moving


In [ ]:
# find the slice with most tumorus
def get_most_tumorus_slice(seg):
    assert seg.ndim==3
    return np.argmax(seg.sum(axis=(2,1)))
# get bounding box on img
def get_bbox(seg):
    assert seg.ndim==3
    nz = seg.nonzero()
    lft = nz[0].min(), nz[1].min(), nz[2].min()
    rgt = nz[0].max(), nz[1].max(), nz[2].max()
    return np.concatenate([lft, rgt])

dct_keys = dict(zip(keys, zip(*seg_pairs)))

def visualize_bbox_img(dct, keys=['seg1', 'img1']):
    sli_num = get_most_tumorus_slice(dct[f'{keys[0]}'][vis_id]>1.5)
    print(sli_num)
    seg = dct[f'{keys[0]}'][vis_id][sli_num][None]
    img = dct[keys[1]][vis_id][sli_num][None]
    bbox = get_bbox(seg>1.5)
    print(bbox)
    img = draw_3d_box_on_vol(img[None], bbox[None])
    save_image(img, f'images/{dir_name}/{keys[1]}.jpg')
    # save seg
    save_image((torch.tensor(seg)>1.5).float(), f'images/{dir_name}/{keys[0]}.jpg')

# visualize_bbox_img(dct_keys, keys=['seg1', 'img1'])
# visualize_bbox_img(dct_keys, keys=['seg2', 'img2'])
# visualize_bbox_img(dct_keys, keys=['warped_seg_moving', 'warped_moving'])

# union of seg
seg1 = dct_keys['seg1'][vis_id]
seg2 = (dct_keys['seg2'][vis_id]==2).astype(np.int)*2
seg = np.maximum(seg1[103], seg2[89])
w_img = dct_keys['warped_moving'][vis_id][98]
w_img[seg==2]=0
# save w_img
save_image(torch.tensor(w_img)/255, f'images/{dir_name}/w_img.jpg')
# save_image((torch.tensor(seg)).float()/2, f'images/{dir_name}/seg.jpg')

### 2D grid + color

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

# visualize each flow in plt subplot
def plot_flow(flows, width, inter_dst):
    le = len(flows[::inter_dst])
    r = (le+width-1)//width
    fig, axes = plt.subplots(r, width, figsize=(width*7, r*5))
    print(r)
    x, y, z = np.meshgrid(np.arange(128), np.arange(128), np.arange(128), indexing='ij')
    u, v, w = np.transpose(flows, (-1, 0, 1, 2))
    fx, fy, fz = u, y+v, z+w
    fff = np.stack((fx, fy, fz), axis=1)[::inter_dst]
    for flow, ax in zip(fff, axes.flat):
        cp = plot_single_flow(flow, fig, ax)
        fig.colorbar(cp, ax=ax)
    fig.tight_layout(pad=0)
    

def plot_single_flow(flow, fig, ax):
    fx, fy, fz = flow
    plot_grid(fz, fy, ax, colors='black', alpha=0.3)
    cp = ax.contourf(fz, fy, fx, cmap='bwr', alpha=0.3)
    ax.axis('off')
    return cp

def plot_grid(x,y, ax=None, **kwargs):
    ax = ax or plt.gca()
    segs1 = np.stack((x,y), axis=2)
    segs2 = segs1.transpose(1,0,2)
    ax.add_collection(LineCollection(segs1, **kwargs))
    ax.add_collection(LineCollection(segs2, **kwargs))
    ax.autoscale()
    return ax

flow = dct['real_flow'][1]
plot_flow(flow, 5, 3)
plt.savefig('plt.png')
# plt.show()
# fig.canvas.draw()

# Now we can save it to a numpy array.
# data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
# data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))

# Unused

point

vector field

In [ ]:
def sample(x, sk):
    return x[::sk, ::sk, ::sk]
x,y,z = np.meshgrid(np.arange(128), np.arange(128), np.arange(128), indexing='ij')
u, v, w = np.transpose(flow, (-1, 0, 1, 2))

ax = plt.figure(figsize=(10,10)).add_subplot(projection='3d')

nu,nv,nw = sample(u+x,24), sample(v+y,24), sample(w+z,24)
# ax.scatter(sample(x, 12), sample(y, 12), sample(z,12), s=1, c='r')
# ax.scatter(nu, nv, nw, s=10, c='r')
for i in range(nu.shape[0]):
    # ax.plot_wireframe(nu[i], nv[i], nw[i], rstride=1, cstride=1, linewidth=1)
    # ax.plot_wireframe(nu[i], nv[i], nw[i], rstride=1, cstride=1, linewidth=1)
    ax.plot_surface(nu[i], nv[i], nw[i], rstride=1, cstride=1, cmap=plt.get_cmap('gist_earth'))
    break
ax.view_init(0, 0)

# ax.quiver(sample(x, 12), sample(y, 12), sample(z,12), sample(u,12), sample(v,12), sample(w,12), length=1, normalize=True, linewidth=5)
plt.show()

Wireframe

In [ ]:
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Grab some test data.
X, Y, Z = axes3d.get_test_data(0.05)

# Plot a basic wireframe.
# ax.plot_wireframe(X, Y, Z, rstride=10, cstride=10)
# ax.plot_wireframe(X+5, Y, Z+10, rstride=10, cstride=10)
ax.quiver(X,Y,Z, length=0.1, normalize=True)

plt.show()

### Utilities

In [ ]:
def cal_area(seg):
    return np.sum(np.sum(seg>1.5, axis=1), axis=1)
plt.plot(cal_area(dct['seg1'][0,...,1]))
# plt.plot(cal_area(p[0]))
plt.show()

In [ ]:
from data_util.liver import Dataset
ds = Dataset('/home/hynx/regis/Recursive-Cascaded-Networks/datasets/liver_mini.json', batch_size=1)
gen = ds.generator('lits')
a = next(gen)
a['id1']

In [8]:
# split train and val
lst = [f"lits/{i}" for i in range(131)]
lst = np.random.permutation(lst)
lst[:100].tolist(), lst[100:].tolist()

(['lits/108',
  'lits/113',
  'lits/129',
  'lits/68',
  'lits/99',
  'lits/4',
  'lits/59',
  'lits/100',
  'lits/10',
  'lits/127',
  'lits/18',
  'lits/62',
  'lits/23',
  'lits/98',
  'lits/88',
  'lits/25',
  'lits/103',
  'lits/45',
  'lits/34',
  'lits/41',
  'lits/31',
  'lits/11',
  'lits/64',
  'lits/110',
  'lits/109',
  'lits/38',
  'lits/117',
  'lits/119',
  'lits/67',
  'lits/1',
  'lits/85',
  'lits/43',
  'lits/90',
  'lits/55',
  'lits/91',
  'lits/125',
  'lits/13',
  'lits/8',
  'lits/72',
  'lits/2',
  'lits/15',
  'lits/37',
  'lits/26',
  'lits/104',
  'lits/80',
  'lits/126',
  'lits/21',
  'lits/94',
  'lits/77',
  'lits/19',
  'lits/35',
  'lits/63',
  'lits/16',
  'lits/66',
  'lits/50',
  'lits/17',
  'lits/54',
  'lits/6',
  'lits/96',
  'lits/95',
  'lits/112',
  'lits/82',
  'lits/86',
  'lits/120',
  'lits/28',
  'lits/114',
  'lits/0',
  'lits/87',
  'lits/105',
  'lits/7',
  'lits/130',
  'lits/57',
  'lits/124',
  'lits/29',
  'lits/14',
  'lits/70',


## Read h5 file

In [2]:
import os
# read hd5f file and get the data
import h5py
import numpy as np
from adet.utils.visualize_niigz import *
# path = '/home/hynx/regis/Recursive-Cascaded-Networks/datasets/sliver_val.h5'
# path = '/home/hynx/regis/Recursive-Cascaded-Networks/datasets/lits_uint8.h5'
path = '/home/hynx/regis/Recursive-Cascaded-Networks/datasets/lits_bkp.h5'
# path = '/home/hynx/regis/Recursive-Cascaded-Networks/datasets/youyi_liver.h5'   
# path = '/home/hynx/regis/Recursive-Cascaded-Networks/datasets/msd_dataset.h5'
# path = '/home/hynx/regis/Recursive-Cascaded-Networks/datasets/lspig_val.h5'
# path = '/home/hynx/regis/Recursive-Cascaded-Networks/datasets/lpba_val.h5'
f = h5py.File(path, 'r')
# print(len(f))
ks = list(f.keys())
print(ks)
print(len(ks))
# for k in ks:
#     print(k, len(f[k].keys()))
print(np.array(f[ks[0]]))

def cal_data():
    for k in f:
        data = f[k]['volume']
        data = np.array(data)
        print(data.dtype)
        visulize_3d(data, save_name='img1.png')
        print(np.unique(data))
        break
def cal_seg():
    for k in f:
        seg = f[k]['segmentation']
        seg = np.array(seg)
        visulize_3d(seg, save_name='seg1.png')
        print(np.unique(seg))
        if (seg>1.5).sum()==0:
            print(k)
        break
cal_data()
cal_seg()

def visualize_every_sample(f, dir):
    for k in f:
        data = np.array(f[k]['volume'])
        seg= np.array(f[k]['segmentation'])
        visulize_3d(data, save_name=f'images/{dir}/{k}_img.png')
        visulize_3d(seg, save_name=f'images/{dir}/{k}_seg.png')
        print(k)
dir = 'sliver'
# makedir
# if not os.path.exists(f'images/{dir}'):
#     os.makedirs(f'images/{dir}')
# visualize_every_sample(f, dir)

f.close()

['0', '1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']
131
['point' 'segmentation' 'volume']
uint8
Visualizing img with shape and type: torch.Size([26, 1, 128, 128]) torch.float32
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  1

In [1]:
import h5py, numpy as np
path = '/home/hynx/regis/Recursive-Cascaded-Networks/datasets/lits_bkp.h5'
f = h5py.File(path, 'r+')
# turn label 255 to 1
def modify_h5():
    for k in f:
        data = f[k]['segmentation']
        data = np.array(data)
        # normalize data to 0-255
        # data = (data - data.min()) / (data.max() - data.min()) * 255
        # binarize seg
        data = data>255/2
        data = data.astype(np.uint8)
        f[k]['segmentation'][...] = data
# modify_h5()     
f.close()

In [ ]:
# import random
# random.shuffle(ks)
# with open('/home/hynx/regis/Recursive-Cascaded-Networks/datasets/lits.txt', 'w') as f:
#     for k in ks:
#         f.write(k + '\n')